# Notebook for making basic PID by separating mu- and pi- based on layers traversed

In [8]:
#imports
import numpy as np
import uproot as up
import pandas as pd
import matplotlib.pyplot as plot

In [9]:
#USER SET VARIABLES

#Set these according to the simulation
select_pdg = 13 #pdg value of particle gun (mu- = 13; pi- = 211)

In [10]:
uproot_path = f"/cwork/rck32/eic/work_eic/root_files/June_10/mu_5GeV_10k.edm4hep.root:events"

events = up.open(uproot_path)

x_pos_branch = events["HcalBarrelHits/HcalBarrelHits.position.x"].array(library='np')
PDG_branch = events["MCParticles.PDG"].array(library='np')

In [11]:
def get_num_layers_traversed(x_pos):
    #This map contains the midpoints of each layer - each layer is 1cm in width, so we can check the 0.5cm on each side to see if a hit was in the layer
    layer_map = [1830.8000, 1841.4000, 1907.5, 1918.1,1984.1999, 1994.8000, 2060.8999,2071.5,2137.6001,2148.1999,2214.3000,2224.8999,2291,2301.6001,2367.6999,2378.3000,2444.3999,2455,2521.1001,2531.6999,2597.8000,2608.3999,2674.5,2685.1001,2751.1999,2761.8000,2827.8999,2838.5]
    for layer_idx in range(len(layer_map)):
        #check if particle hit within layer
        if((x_pos >= layer_map[layer_idx] - 5) and (x_pos <= layer_map[layer_idx] + 5)):
            continue
        elif(layer_idx == 0):
            #if didn't hit first layer, skip event and make note
            return -1
        else:
            #if particle didn't hit layer, must have stopped previous layer
            return layer_idx - 1
    return -2

In [12]:
#initialize list that we will fill with # of layers traversed for each event
layers_traversed = []

#loop over each event
for event_idx in range(len(x_pos_branch)):
    event_x_pos = x_pos_branch[event_idx]
    #for each event, loop over the particles to find mu/pi
    for particle_idx in range(len(event_x_pos)):
        particle_pdg = PDG_branch[event_idx][particle_idx]
        if(particle_pdg == select_pdg):
            current_x_pos = event_x_pos[particle_idx]
            num_layers_traversed = get_num_layers_traversed(current_x_pos)
            if(num_layers_traversed == -1):
                print(f"Event #{event_idx} did not hit first layer... skipping")
                continue
            elif(num_layers_traversed == -2):
                print(f"Error with event #{event_idx}... skipping")
                continue
            layers_traversed.append(num_layers_traversed)#add to list

IndexError: index 1 is out of bounds for axis 0 with size 1

In [14]:
PDG_branch[0]

array([13], dtype=int32)